! jt -r

In [3]:
from importlib import reload
reload(aux)
reload(con)
reload(dt)
reload(info)

<module 'src.info' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\info.py'>

In [4]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime,date
import src.info as info
import src.in_out as io
import numpy as np
from src.density import Density, draw
import itertools

# Hazırlık Aşaması

**data** klasöründeki tüm dosya yolları alınıyor
```python
>>> path_list
Out []:
 ['C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6C_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6C_U8.xlsx']
```

In [5]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

Ürün adları sırayla alınıyor
```py
>>> product_names
Out[]:
['6AU8', '6BU8', '6CU8', '6AU8', '6BU8', '6CU8']```

In [6]:
product_names = [info.get_productName(path) for path in path_list]

Tarihler sırayla alınıyor
```py
>>> date_list
Out[]:
[Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00')]
```

In [7]:
date_list = [info.get_productDate(path) for path in path_list]

Hangi sütunlardaki verileri alacağımızı aşağıdaki listede belirityoruz

In [8]:
cols= ["Time","BID price","ASK price"]

Dosyalardan veriler dataframe olarak okunuyor ve listeye atılıyor.
Okunan_veriler'in yapısı: <br/>
```py
list(pd.DataFrame,pd.DataFrame.....)
```

In [9]:
okunan_veriler = [pd.read_excel(path,usecols=cols) for path in path_list]

Her bir DataFrame'deki sütunlar yeniden adlandırılıyor

In [10]:
columns=['time','bid_price','ask_price']

In [11]:
for df in okunan_veriler:
    df.columns = columns

In [12]:
time_stamp = okunan_veriler[0].time

# Mid Price Hesaplama ve Saatlik Bölme 

hesaplanan her bir mid_price **list_of_mid_price** adlı listeye atılıyor. Listenin yapısı aşağıdaki gibidir:
```py
list(pd.Series,pd.Series,pd.Series...)
```

In [13]:
list_of_mid_price = list(map(info.get_mid_price,okunan_veriler))

Bulunan mid price hiyerarşik indeksleme ile tek DataFrame'de birleştiriliyor.

In [14]:
arrays = [date_list,product_names]

In [15]:
df_mp = pd.DataFrame(data=list_of_mid_price, index = arrays)

Bu dataframe zaman sütunu eklenip saatlik olarak bölünüyor.

In [16]:
df_mp_t = df_mp.T

In [17]:
df_mp_t['time'] = time_stamp

```py
>>> len(list_of_hourly_mp)
Out[]: 23
>>> df.columns.levels
Out[]:FrozenList([[2018-09-06 00:00:00, 2018-09-07 00:00:00, 'time'], ['6AU8', '6BU8', '6CU8', '']])
```

In [18]:
list_of_hourly_mp = info.split_df(df_mp_t,23)

----

# Ürünler Eşleştiriliyor

```python
>>> non_duplicate_dates
Out[]:[Timestamp('2018-09-06 00:00:00'), Timestamp('2018-09-07 00:00:00')]
>>> non_duplicate_names
Out[]:['6AU8', '6BU8', '6CU8']
>>> paired_products
Out[]: [('6AU8', '6BU8'), ('6AU8', '6CU8'), ('6BU8', '6CU8')]
```

In [19]:
non_duplicate_dates = list(dict.fromkeys(date_list)) #tekrar eden tarihler kaldırıldı.

In [20]:
non_duplicate_names = list(dict.fromkeys(product_names)) #tekrar eden ürünler kaldırıldı.

In [21]:
paired_products = list(itertools.combinations(non_duplicate_names, 2)) #2'li kombinasyon kullanılarak ürünler eşleştiriliyor.

----

# Spread Hesaplanıyor

**list_of_hourly_mp** da hiyerarşi şu şekildedir: <br/>
* Önce saate erişiriz
* İkinci olarak tarihe erişiriz
* Son olarak ürüne erişiriz <br/>
**Buna göre spread hesaplamak için aşağıdaki yol izlenir:** <br/>
    1. Her bir saate eriş
    2. Her bir tarihe eriş
    3. Pairleri al
    4. *spread_hesapla(pair): return pd.Series*
    

In [22]:
values={'a_PNLTICK':10,
'a_TICKSIZE':0.0001,
'b_PNLTICK':6.25,
'b_TICKSIZE':0.0001}

In [23]:
def spread_al(df):
    spread_dict = {}
    for date in non_duplicate_dates:
        for pair in paired_products:
            mp_1 = df[pd.Timestamp(date)][pair[0]]
            mp_2 = df[pd.Timestamp(date)][pair[1]]
            spread_dict[(date,pair[0]+"_"+pair[1])] = aux.find_spread(mp_1,mp_2,values)
    new_df = pd.DataFrame(spread_dict)
    new_df['time'] = df.time
    return new_df

In [24]:
spread = [spread_al(df) for df in list_of_hourly_mp]

----

# Change Hesaplanıyor

In [25]:
pairs = [pair[0]+'_'+pair[1] for pair in paired_products]

In [26]:
def change_al(df):
    change_dict = {}
    for date in non_duplicate_dates:
        for pair in pairs:
            sub_df = df[date][pair]            
            change_dict[(date,pair)] = aux.find_change(sub_df)
    new_df = pd.DataFrame(change_dict)
    new_df['time'] = df.time
    return new_df

In [27]:
change_list = [change_al(df) for df in spread]

In [55]:
change_list[0]

2018-09-06 00:00:00                      2018-09-07 00:00:00            \
               6AU8_6BU8 6AU8_6CU8  6BU8_6CU8           6AU8_6BU8 6AU8_6CU8   
1              31250.000       NaN -50000.000                 NaN       NaN   
2                 -5.000    -1.875      3.125              -5.000    -1.875   
3                 -3.125     0.000      5.000              -3.125     0.000   
4                 -3.125     0.000      5.000              -3.125     0.000   
5                  3.750     3.750      3.750               3.750     3.750   
6                  3.125     0.000     -5.000               3.125     0.000   
7                  0.000     0.000      0.000               0.000     0.000   
8                  3.125     0.000     -5.000               3.125     0.000   
9                 -3.125     0.000      5.000              -3.125     0.000   
10                 3.125     0.000     -5.000               3.125     0.000   
11                 0.000     0.000      0.000               0.000     0.000   
12                 0.000     0.000      0.000               0.000     0.000   
13                 0.000     0.000      0.000               0.000     0.000   
14                -6.250     0.000     10.000              -6.250     0.000   
15                 0.000     0.000      0.000               0.000     0.000   
16                 3.125     0.000     -5.000               3.125     0.000   
17                 3.125     0.000     -5.000               3.125     0.000   
18                -6.250     0.000     10.000              -6.250     0.000   
19                 0.000     0.000      0.000               0.000     0.000   
20                 0.000     0.000      0.000               0.000     0.000   
21                 0.000     0.000      0.000               0.000     0.000   
22                 0.000     0.000      0.000               0.000     0.000   
23                 0.000     0.000      0.000               0.000     0.000   
24                 0.000     0.000      0.000               0.000     0.000   
25                 0.000     0.000      0.000               0.000     0.000   
26                 0.000     0.000      0.000               0.000     0.000   
27                 0.000     0.000      0.000               0.000     0.000   
28                 0.000     0.000      0.000               0.000     0.000   
29                 0.000     0.000      0.000               0.000     0.000   
30                 0.000     0.000      0.000               0.000     0.000   
...                  ...       ...        ...                 ...       ...   
3422               0.000       NaN      0.000               0.000       NaN   
3423               0.000       NaN      0.000               0.000       NaN   
3424               0.000       NaN      0.000               0.000       NaN   
3425               0.000       NaN      0.000               0.000       NaN   
3426               0.000       NaN      0.000               0.000       NaN   
3427               0.000       NaN      0.000               0.000       NaN   
3428               0.000       NaN      0.000               0.000       NaN   
3429               0.000       NaN      0.000               0.000       NaN   
3430              -3.125       NaN      5.000              -3.125       NaN   
3431               0.000       NaN      0.000               0.000       NaN   
3432               0.000       NaN      0.000               0.000       NaN   
3433               0.000       NaN      0.000               0.000       NaN   
3434               0.000       NaN      0.000               0.000       NaN   
3435               0.000       NaN      0.000               0.000       NaN   
3436               0.000       NaN      0.000               0.000       NaN   
3437               3.125       NaN     -5.000               3.125       NaN   
3438              -3.125       NaN      5.000              -3.125       NaN   
3439               3.125       NaN     -5.000          

---

# Duration ve Amplitude Hesaplanıyor

Hesaplama ile birlikte birleştirme de yapılıyor.
Birleştirme koşulu: <br/>
1. Ürünler aynı olacak
2. Tarihler farklı olacak
3. Saat dilimi aynı olacak

In [28]:
def unnamed1(df):
    lengt = df.shape[1]
    df_list = []
    for i in range(lengt):
        df_list.append(aux.find_duramp(df.iloc[:,1]))
    return pd.concat(df_list).dropna()

In [29]:
def unnamed2(df):
    new_df = duzenle(df)
    product = {}
    for pair in pairs:
        product[pair] = unnamed1(new_df[pair])
    return product

In [30]:
def duzenle(df):
    new_df = df.set_index('time')
    new_df = new_df.T.reset_index(level=0,drop=True).T
    return new_df

In [31]:
duramp = [unnamed2(change) for change in change_list]

---

# Medyanı baz alarak verileri yeniden düzenliyoruz

**pozitif ve negatif medyan, duration ve amplitude için ayrı ayrı baz alınarak hesaplanmıştır**
```py
>>> edited_data.keys()
Out[]: dict_keys(['6AU8_6BU8', '6AU8_6CU8', '6BU8_6CU8'])
```
Bir ürüne ait tüm veriye erişmek için:
```py
edited_data['6AU8_6BU8']
```
Bir ürünün belirli bir saatine ait veriye erişmek için:
```py
edited_data['6AU8_6BU8'].loc[time(18)]
```

In [38]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

In [40]:
edited_data = {}

In [41]:
for pair in pairs:
    con_list = []
    for i in range(23):
        current = duramp[i][pair]
        con_df = con.single_scan(current)
        con_list.append(con_df)
    edited_data[pair] = pd.concat(con_list,keys=hour_series)